In [9]:
import os
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
import shutil
import os
import sys
import shutil
import wandb
%reload_ext autoreload
%autoreload 2
sys.path.append('../')
from Helper_Modules import resize
from Helper_Modules import data_setups, utils, model_setups
from Helper_Modules.dataset_creation import create_organ_dataset_1, create_organ_dataset_2, find_cells_by_keyword
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
file_path = "..\\data\\allData\\Data_Map.xlsx" 


# create a folder for the first collection of the data

In [6]:
test_ear    =   find_cells_by_keyword(file_path, {3:"Ear"   , 4:"first", 5:"test" })
train_ear   =   find_cells_by_keyword(file_path, {3:"Ear"   , 4:"first", 5:"train"} )
test_nose   =   find_cells_by_keyword(file_path, {3:"Nose"  , 4:"first", 5:"test" })
train_nose  =   find_cells_by_keyword(file_path, {3:"Nose"  , 4:"first", 5:"train"} )
test_vocal  =   find_cells_by_keyword(file_path, {3:"vocal" , 4:"first", 5:"test" })
train_vocal =   find_cells_by_keyword(file_path, {3:"vocal" , 4:"first", 5:"train"} )

len(test_ear), len(train_ear), len(test_nose), len(train_nose), len(test_vocal), len(train_vocal)

(37, 220, 128, 307, 78, 111)

In [8]:
create_organ_dataset_1("organ_0", test_ear, train_ear, test_nose, train_nose, test_vocal, train_vocal)

# V0
noisy data set, uncropped, and imbalanced

In [12]:
Organ_0_path = "data\\organ_0"
Organ_0_config = dict(dataset_dir=Organ_0_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.Resize((224,224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_0",
                      description="noisy data set, uncropped, and imbalanced")
Organ_0_config

{'dataset_dir': 'data\\organ_0',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn),
  ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_0',
 'description': 'noisy data set, uncropped, and imbalanced'}

In [14]:
data_setups.load_and_log_IF(Organ_0_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: elsharkawi99 (d-ml). Use `wandb login --relogin` to force relogin


# V1
train dataset is manually cropped

In [15]:
shutil.copytree('data/organ_0/', 'data/organ_1')

'data/organ_1'

In [16]:
#all the ear images are 1920x1080, the position of the image is fixed
image_path_list = list(Path("data/organ_1").glob("train/Ear/*.png"))
new_imgs = []
for path in image_path_list:
    with Image.open(path) as img:
        width, height = img.size
        left = (width - 770)/2
        top = (height - 770)/2
        right = (width + 770)/2
        bottom = (height + 770)/2
        img = img.crop((left, top, right, bottom))
        new_imgs.append(img)
        img.save(path)

In [17]:
# only strobo_1 vidio has a black frame
image_path_list = list(Path("data/organ_1").glob("train/VocalFolds/strobo_1*.png"))
new_imgs = []
for path in image_path_list:
    with Image.open(path) as img:
        width, height = img.size
        left = (width - 1280)/2
        top = (height - 1080)/2
        right = (width + 1610)/2
        bottom = (height + 1080)/2
        img = img.crop((left, top, right, bottom))
        new_imgs.append(img)
        img.save(path)

In [18]:
Organ_1_path = "data\\organ_1"
Organ_1_config = dict(dataset_dir=Organ_1_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.Resize((224,224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_1",
                      description="noisy data set, train cropped, test not cropped, and imbalanced")
Organ_1_config

{'dataset_dir': 'data\\organ_1',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn),
  ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_1',
 'description': 'noisy data set, train cropped, test not cropped, and imbalanced'}

In [19]:
data_setups.load_and_log_IF(Organ_1_config)

# V2
train and test datasets are uniformly preproccesed 

In [21]:
shutil.copytree('data/organ_0/', 'data/organ_2')

'data/organ_2'

In [28]:
dataset_2_paths = list(Path("data\\organ_2").glob("*\\*\\*.png"))
len(dataset_2_paths)

881

In [29]:
for im in dataset_2_paths:
    resize.to_square_save(im, os.path.join('data', 'organ_2'))

In [30]:
Organ_2_path = "data\\organ_2"
Organ_2_config = dict(dataset_dir=Organ_2_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_2",
                      description="noisy data set, all cropped, and imbalanced")
Organ_2_config

{'dataset_dir': 'data\\organ_2',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_2',
 'description': 'noisy data set, all cropped, and imbalanced'}

In [32]:
data_setups.load_and_log_IF(Organ_2_config)

# functions to create a dataset and equalizes the classes
For creating the seccond collection of the data, a new function has been written. To make it easier to look over the data before choosing the splits

Helper_Modules.dataset_creation.create_organ_dataset_n()

This fuction does the following:
 - creates a copy from the original data source and stores iit with the same name as the dataset_base
 - calculates a train test split of 5:1. ((The split is based on the class with less images))
 - prints all the patient directories for the user and asks for the names of the patients to use as test data
 - shuffles the list of the choosen images and strts moving them to the test directories until the max split number is reached and delets the rest, so it's guranteed these patients will not be used in the train dataset
 - calculate the ´min_image_count´ among all the classes again (while chossing test patients, more images might have been deleted) 
 - after shuffling the list of all the images in each class, so images from each patient are used, the ´min_image_count´will be moved to the train dirs


# V3: imbalanced dataset
cropped, noisy, imbalanced
test patients: 
- Ear_6_1,Ear_6_2
- Nose_3_2
- Vocal_1,Vocal_4

In [10]:
create_organ_dataset_2(Path("data\\Organ_3"), ["Ear", "Nose", "VocalFolds"], Path("..\\data\\allData"), False)

70 in Ear_1_1
57 in Ear_1_2
32 in Ear_2_1
61 in Ear_2_2
37 in Ear_3_1
56 in Ear_4_1
66 in Ear_4_2
97 in Ear_5_1
71 in Ear_5_2
38 in Ear_6_1
69 in Ear_6_2
65 in Ear_7_1
40 in Ear_7_2

------------------------------------------

78 in Nose_1_1
50 in Nose_1_2
102 in Nose_2_1
81 in Nose_2_2
124 in Nose_3_2
101 in Nose_4_1
126 in Nose_4_2
100 in Nose_5_1
110 in Nose_5_2
100 in Nose_6_1
76 in Nose_6_2

------------------------------------------

51 in Vocal_1
36 in Vocal_2
42 in Vocal_3
60 in Vocal_4
43 in Vocal_5
57 in Vocal_6
75 in Vocal_7
55 in Vocal_8
71 in Vocal_9

------------------------------------------

min image in one class: 490
select 98 images for the test dataset and delete it from the base dir. In case you did't do it
select 392 images for the test dataset and delete it from the base dir. In case you did't do it
inter the test directories for each organ splited by ","


In [39]:
organ_3_paths = list(Path("data/organ_3/").glob("*/*/*.png"))
len(organ_3_paths)

2297

In [40]:
for im in organ_3_paths:
    resize.to_square_save(im, "data/organ_3/")

In [41]:
organ_3_path = "data/organ_3/"
organ_3_config = dict(dataset_dir=organ_3_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_3",
                      description="more images, noisy, and imbalanced")
organ_3_config

{'dataset_dir': 'data/organ_3/',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_3',
 'description': 'more images, noisy, and imbalanced'}

In [42]:
data_setups.load_and_log_IF(organ_3_config)

# V4: balanced noisy dataset
balanced noisy dataset
test patients: ear 6
nose 3
vocal 1,4

In [47]:
create_organ_dataset_n(Path("data/organ_4"), ["Ear", "Nose", "VocalFolds"], Path("../data/allData"))

70 in Ear_1_1
57 in Ear_1_2
32 in Ear_2_1
61 in Ear_2_2
37 in Ear_3_1
56 in Ear_4_1
66 in Ear_4_2
97 in Ear_5_1
71 in Ear_5_2
38 in Ear_6_1
69 in Ear_6_2
65 in Ear_7_1
40 in Ear_7_2

------------------------------------------

78 in Nose_1_1
50 in Nose_1_2
102 in Nose_2_1
81 in Nose_2_2
124 in Nose_3_2
101 in Nose_4_1
126 in Nose_4_2
100 in Nose_5_1
110 in Nose_5_2
100 in Nose_6_1
76 in Nose_6_2

------------------------------------------

51 in Vocal_1
36 in Vocal_2
42 in Vocal_3
60 in Vocal_4
43 in Vocal_5
57 in Vocal_6
75 in Vocal_7
55 in Vocal_8
71 in Vocal_9

------------------------------------------

min image in one class: 490
select 98 images for the test dataset and delete it from the base dir. In case you did't do it
select 392 images for the test dataset and delete it from the base dir. In case you did't do it
inter the test directories for each organ splited by ","


In [48]:
organ_4_paths = list(Path("data/organ_4/").glob("*/*/*.png"))
len(organ_4_paths)

1431

In [49]:
for im in organ_4_paths:
    resize.to_square_save(im, "data/organ_4")

In [50]:
Organ_4_path = "data/organ_4/"
Organ_4_config = dict(dataset_dir=Organ_4_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_4",
                      description="balanced noisy dataset")
Organ_4_config

{'dataset_dir': 'data/organ_4/',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_4',
 'description': 'balanced noisy dataset'}

In [51]:
data_setups.load_and_log_IF(Organ_4_config)

# V5: train and test clean
noise is filtered using the trained quality-classifier (tr_v0_64_1e5:v0)

## cleaning the base dataset from noise using the quality model:tr_v0_64_1e5:v0

In [3]:
model = model_setups.efficientNet_b0(False, 2, device)
tr_v0_64_1e5 = utils.download_model("Quality-Classifier-1", "tr_v0_64_1e5:v0", "trained_model.pth", model)
auto_transforms = [
    transforms.ToTensor(), # this includes scaling the pixel values 0..1
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]

downloading the model: tr_v0_64_1e5:v0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: elsharkawi99 (d-ml). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [4]:
all_data_paths = list(Path("../data/allData").glob("*/*/*.png"))
len(all_data_paths)

2297

In [5]:
data_setups.sort_out_noise(auto_transforms, tr_v0_64_1e5, "tr_v0_64_1e5", device, ["bad", "good"], all_data_paths, True, 0.5)

In [12]:
create_organ_dataset_n(Path("data/Organ_5"), ["Ear", "Nose", "VocalFolds"], Path("../data/tr_v0_64_1e5/allData-clean-0.5"))

62 in Ear_1_1
37 in Ear_1_2
11 in Ear_2_1
43 in Ear_2_2
28 in Ear_3_1
56 in Ear_4_1
65 in Ear_4_2
56 in Ear_5_1
36 in Ear_5_2
37 in Ear_6_1
64 in Ear_6_2
50 in Ear_7_1
32 in Ear_7_2

------------------------------------------

54 in Nose_1_1
50 in Nose_1_2
79 in Nose_2_1
73 in Nose_2_2
95 in Nose_3_2
2 in Nose_4_1
32 in Nose_4_2
4 in Nose_5_1
34 in Nose_5_2
4 in Nose_6_1
38 in Nose_6_2

------------------------------------------

51 in Vocal_1
36 in Vocal_2
42 in Vocal_3
53 in Vocal_4
43 in Vocal_5
57 in Vocal_6
75 in Vocal_7
55 in Vocal_8
68 in Vocal_9

------------------------------------------

min image in one class: 465
select 93 images for the test dataset and delete it from the base dir. In case you did't do it
select 372 images for the test dataset and delete it from the base dir. In case you did't do it
inter the test directories for each organ splited by ","


In [13]:
organ_5_paths = list(Path("data/organ_5/").glob("*/*/*.png"))
len(organ_5_paths)

1389

In [14]:
for im in organ_5_paths:
    resize.to_square_save(im, "data/Organ_5/")

In [15]:
organ_5_path = "data/organ_5"
organ_5_config = dict(dataset_dir=organ_5_path,
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_5",
                      description="all the classes are equalized, the noise has been cleaned")
organ_5_config

{'dataset_dir': 'data/organ_5',
 'project_name': 'Organ-Classifier-1',
 'transforms_': [ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
 'name': 'organ_5',
 'description': 'all the classes are equalized, the noise has been cleaned'}

In [16]:
data_setups.load_and_log_IF(organ_5_config)

# logg EffNet to WandB

In [61]:
model_config = dict(project_name="Organ-Classifier-1",
                    model_name="efficientNet_b0",
                    description="efficientnet unfrozen",
                    freeze_layers=False,
                    num_classes=3)

model_ = model_setups.efficientNet_b0(model_config['freeze_layers'],model_config['num_classes'], device)
model_initialized = model_setups.build_model_and_log(model_,model_config)

OSError: [WinError 1314] A required privilege is not held by the client: 'd:\\BachelorArbeit\\GitClone\\2023_endoscopy_classification_elsharkawi\\Organ-Classifier\\initialized_model.pth' -> 'd:\\BachelorArbeit\\GitClone\\2023_endoscopy_classification_elsharkawi\\Organ-Classifier\\wandb\\run-20230721_135642-pvvehlc1\\files\\initialized_model.pth'

# download dataset v4 and v5 from WandB

In [2]:
train_dataset_v4, test_dataset_v4 = utils.load_data_wandb({"project_name":"Organ-Classifier-1",
                                                     "dataset_name":"organ_4:v0",
                                                     "download":True})

train_dataset_v5, test_dataset_v5 = utils.load_data_wandb({"project_name":"Organ-Classifier-1",
                                                     "dataset_name":"organ_5:v0",
                                                     "download":True})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: elsharkawi99 (d-ml). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact organ_4:v0, 821.72MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:3.8


wandb: Downloading large artifact organ_5:v0, 797.60MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:22.2


# V6 train clean test noisy
- train from V5 
- test from V4

copy the .pt files from the downloaded datasets under artifacts/datasets and place it under data/organ_6

In [12]:
def copy_datasets(dest_path, train_path, test_path):
    os.makedirs(dest_path, exist_ok=True)
    shutil.copy(train_path, dest_path)
    shutil.copy(test_path, dest_path)

In [13]:
copy_datasets("data/organ_6", "artifacts/datasets/organ_5:v0/train.pt", "artifacts/datasets/organ_4:v0/test.pt")

In [14]:
# v6 consists of train.pt from v5 and test.pt from v4. # The dictionaries are here just to log into WandB, but the datasets are already processed 
v6_dict = {"train_dataset":{"Ear":"370", "Nose":"370", "sum":"1110","VocalFolds":"370"},
 "test_dataset":{"Ear":"98", "Nose":"98", "sum":"294","VocalFolds":"98"}}
organ_6_config = dict(dataset_dir="data/organ_6/",
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_6",
                      description="all the classes are equalized, the noise has been cleaned just from the train")
wandb.init(project="Organ-Classifier-1", config=organ_6_config, job_type="load_data")

artifact = wandb.Artifact(name=organ_6_config["name"],
                              type="dataset",
                              description=organ_6_config["description"],
                              metadata=v6_dict)
artifact.add_dir("data/organ_6/")
wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./data/organ_6)... Done. 7.4s


# V7 train noisy test clean
- train from V4 
- test from V5

copy the .pt files from the downloaded datasets under artifacts/datasets and place it under data/organ_7

In [15]:
copy_datasets("data/organ_7", "artifacts/datasets/organ_4:v0/train.pt", "artifacts/datasets/organ_5:v0/test.pt")

In [16]:
# v7 consists of train.pt from v4 and test.pt from v5
v7_dict = {"train_dataset":{"Ear":"379", "Nose":"379", "sum":"1137","VocalFolds":"379"},
 "test_dataset":{"Ear":"93", "Nose":"93", "sum":"279","VocalFolds":"93"}}
organ_7_config = dict(dataset_dir="data/organ_7/",
                      project_name="Organ-Classifier-1",
                      transforms_=[transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])],
                      name="organ_7",
                      description="all the classes are equalized, the noise has been cleaned just from the test")
wandb.init(project="Organ-Classifier-1", config=organ_7_config, job_type="load_data")

artifact = wandb.Artifact(name=organ_7_config["name"],
                              type="dataset",
                              description=organ_7_config["description"],
                              metadata=v7_dict)
artifact.add_dir("data/organ_7/")
wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./data/organ_7)... Done. 7.5s
